In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv

from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [9]:

import random
from datasets import load_dataset, Dataset
import json
import jsonschema
from openai.types.chat.chat_completion import ChatCompletion
import os

domain_search_schema = {
    "type": "object",
    "properties": {
        "entity_name": {
            "type": "string",
            "description": "The name of the primary entity to search for, e.g. 'medical equipment'"
        },
        "name": {
            "type": "string",
            "description": "The name of the search query containing the name of the entity to search for, e.g. 'search medical equipment'"
        },
        "description": {
            "type": "string",
            "description": "A detailed description of the search, when to use it, what it does and how to interpret the results"
        },
        "query_schema": {
            "type": "object",
            "description": "The schema of the query",
            "properties": {
                "type": {
                    "type": "string"
                },
                "description": {
                    "type": "string"
                },
                "properties": {
                    "type": "object",
                },
            },
            "additionalProperties": False,
            "required": ["type", "properties"]
        },
        "result_schema": {
            "type": "object",
            "description": "The schema of the result usually containing a list of entities with their properties, as well as additional metadata",
            "properties": {
                "type": {
                    "type": "string"
                },
                "properties": {
                    "type": "object",
                },
            },
            "additionalProperties": False,
            "required": ["type", "properties"]
        },
    },
    "required": ["name", "entity_name", "query_schema", "result_schema", "description"]
}

def generate_domain_actions(entry):
    results = []

    available_models = [
        "mistralai/mistral-nemo",
        #"mistralai/mixtral-8x7b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mistral-small",
        #"mistralai/mistral-medium",
        "mistralai/mistral-large"
    ]

    model = random.choice(available_models)

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']}. A domain specific search is a way to query historic or current information about the state of relevant entities in that domain. These search queries are very complex and offer a varietry of search options, often including range filter, selectable enums, units of measurement, dates and fuzzy or semantic string comparison. The result contains a result list with everything a {entry['profession']} needs to know about the state of the entities in the domain."
        }, {
            "role": "user",
            "content": f"Generate a complex search query schema and its result schema for a single entity for the profession {entry['profession']} in the process {entry['process']}. The entity should be something the given process revolves around, e.g. 'medical records' or 'students', in an abstract manner. Use the tool call generate_search to generate the search and result schema."
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": "generate_search",
                "parameters": domain_search_schema,
            },
        }],
        tool_choice="required",
        max_tokens=8191,
        temperature=1.2,
        extra_body={
            "provider": {
                "require_parameters": True,
            },
        }
    )

    print(openrouter_response)

    domain_search = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)

    jsonschema.Draft7Validator.check_schema(domain_search["query_schema"])
    jsonschema.Draft7Validator.check_schema(domain_search["result_schema"])

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "name": domain_search["name"],
        "entity_name": domain_search["entity_name"],
        "description": domain_search.get("description", ''),
        "query_schema": domain_search["query_schema"],
        "result_schema": domain_search["result_schema"],
    })
    
    return results


processes = load_dataset("lucaelin/generic_processes_v1", split="train")
domain_actions = processes.shuffle().map(
    sane_batch_map(generate_domain_actions, {"profession": [], "process": [], "name": [], "entity_name": [], "description": [], "query_schema": [], "result_schema": []}), 
    batched=True, 
    batch_size=4, 
    num_proc=4, 
    remove_columns=processes.column_names
)
#domain_actions.sort(["profession", "process", "name"]).to_list()
domain_actions.sort(["profession", "process", "name"]).push_to_hub("lucaelin/generic_domain_searches_v1")

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

ChatCompletion(id='gen-1729799294-itPIZaxCWTj6oiYS3YgJ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='8VzFcsUJV', function=Function(arguments='{"query_schema": {"type": "object", "properties": {"entity_name": {"type": "string", "description": "The name of the entity to search for, e.g. \'medical equipment\'}"}}, "description": "The type of search to be conducted"}, "name": "Radiologic Technologist Equipment Maintenance / Request"}', name='generate_search'), type='function', index=0), ChatCompletionMessageToolCall(id='PXAD21DXS', function=Function(arguments='{"result_schema": {"type": "object", "properties": {"results": {"type": "array", "description": "A list of queried entities", "items": {"type": "object", "properties": {"entity_name": {"type": "string", "description": "The name of the entity"}, "entityId": {"

Map (num_proc=4):   0%|          | 4/1000 [00:23<1:38:17,  5.92s/ examples]

ChatCompletion(id='gen-1729799310-NJMXej7CSwccRG5jL0ZU', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='ltdk7sAzq', function=Function(arguments='{"entity_name": "blueprint", "name": "search blueprints", "description": "This search allows Brickmasons & Blockmasons to find blueprints based on various criteria such as size, complexity, material and date of creation. The search results provide a detailed list of blueprints that meet the specified criteria, including information on the design, materials, and dimensions.", "query_schema": {"type": "object", "properties": {"size": {"type": "number", "description": "The size of the blueprint in square feet"}, "complexity": {"type": "string", "enum": ["simple", "moderate", "complex"], "description": "The level of complexity of the blueprint"}, "material": {"type": "string"

Map (num_proc=4):   1%|          | 8/1000 [00:25<44:52,  2.71s/ examples]  

...

ChatCompletion(id='gen-1729801722-Lk859nJ0Nbw6AHuYsNOq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='UkeC3Jus6', function=Function(arguments='{"entity_name": "Communication", "name": "search Communication", "description": "A complex search query schema and its result schema for a single entity for the profession Construction Manager in the process Communication.", "query_schema": {"type": "object", "properties": {"type": {"type": "string"}, "description": {"type": "string"}, "properties": {"type": "object", "properties": {"communication_type": {"type": "string", "enum": ["Email", "Phone Call", "Meeting", "Letter"]}, "start_date": {"type": "string", "format": "date"}, "end_date": {"type": "string", "format": "date"}, "participants": {"type": "array", "items": {"type": "string"}}}}, "result_schema": {"type": "obj

Map (num_proc=4): 100%|█████████▉| 998/1000 [40:54<00:09,  4.57s/ examples]

ChatCompletion(id='gen-1729801749-fE5xkqvyIGHe1WW2PhTF', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='vr2IZIX2c', function=Function(arguments='{"name": "search massage therapy client records", "entity_name": "client records", "description": "A search to retrieve massage therapy client records containing client\'s history, current conditions, treatments, and notes. Queries the specified parameters to filter and retrieve relevant information.", "query_schema": {"type": "object", "properties": {"client_id": {"type": "string", "description": "Unique identifier of the client"}, "date_range": {"type": "object", "properties": {"start_date": {"type": "string", "format": "date"}, "end_date": {"type": "string", "format": "date"}}, "description": "Range of dates to search within"}, "condition": {"type": "string", "descript

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_domain_actions_v1/commit/7e1487c22d0a3cdced3bd0aa67d799845df40fc7', commit_message='Upload dataset', commit_description='', oid='7e1487c22d0a3cdced3bd0aa67d799845df40fc7', pr_url=None, pr_revision=None, pr_num=None)